# Neural Machine Translation Example

In [1]:
# Install TensorFlow and also our package via PyPI
!pip install tensorflow-gpu==2.0.0
!pip install headliner

     |████████████████████████████████| 380.8MB 61kB/s 
     |████████████████████████████████| 3.8MB 30.8MB/s 
     |████████████████████████████████| 450kB 40.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=15982130e821474e44f06e8d00750b762f5f468f7502c231c6fbbadf753c61da
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: gast 0.3.2
    Uninstalling gast-0.3.2:
      Successfully uninstalled gast-0.3.2
  Found existing installati

In [2]:
# Download the German-English sentence pairs
!wget http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip
!head deu.txt

--2019-10-04 09:17:09--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4541707 (4.3M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   4.33M  5.36MB/s    in 0.8s    

2019-10-04 09:17:10 (5.36 MB/s) - ‘deu-eng.zip’ saved [4541707/4541707]

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              
Hi.	Hallo!
Hi.	Grüß Gott!
Run!	Lauf!
Wow!	Potzdonner!
Wow!	Donnerwetter!
Fire!	Feuer!
Help!	Hilfe!
Help!	Zu Hülf!
Stop!	Stopp!
Wait!	Warte!


In [0]:
# Create the dataset but only take a subset for faster training
import io

def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]
    return zip(*word_pairs)

eng, ger = create_dataset('deu.txt', 30000)
data = list(zip(ger, eng))

In [0]:
# Split the dataset into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=100)

In [5]:
# Define the model and train it
from headliner.trainer import Trainer
from headliner.model.summarizer_attention import SummarizerAttention

summarizer = SummarizerAttention(lstm_size=1024, embedding_size=256)
trainer = Trainer(batch_size=64, 
                  steps_per_epoch=100, 
                  steps_to_log=20, 
                  max_output_len=10, 
                  model_save_path='/tmp/summarizer')
trainer.train(summarizer, train, num_epochs=10, val_data=test)

training a bare model, initializing preprocessing...
vocab encoder: 7551, vocab decoder: 4715, start training loop...
epoch 0, batch 20, logs: {'loss': 3.1557514667510986}
epoch 0, batch 40, logs: {'loss': 2.48339581489563}
epoch 0, batch 60, logs: {'loss': 2.7589330673217773}
epoch 0, batch 80, logs: {'loss': 2.0260400772094727}
epoch 0, batch 100, logs: {'loss': 2.1709651947021484}

(input) Warum bist du zu spät gekommen? 
(target) Why were you late? 
(prediction) you you ? <end>


(input) Benehmt euch nicht daneben! 
(target) Don't misbehave. 
(prediction) you is . <end>


(input) Ich kann auf dich warten. 
(target) I can wait for you. 
(prediction) i i have have . <end>


(input) Tom starb an Krebs. 
(target) Tom died of cancer. 
(prediction) tom is . <end>


(input) Mache bitte mein Bett zurecht! 
(target) Please make my bed. 
(prediction) i have . <end>

loss_val improved from None to 2.345742702484131, saving summarizer to /tmp/summarizer
epoch 1, batch 120, logs: {'loss': 1.932

In [6]:
# Do some prediction
summarizer.predict('Hallo mir geht es gut.')

"hi , i'm fine . <end>"